# qmrpy simulation demo

このノートブックは、qMRLabのシミュレーション系（SingleVoxel / SimVary / SimRnd / SimProtocolOpt: CRLB）をqmrpy側で踏襲した関数が、どのように動作するかを最小例で確認するデモです。

- 対象API: `qmrpy.sim`
- 実行想定: `uv run --locked jupyter lab` など（qmrpyがimportできる環境）


In [ ]:
# Ensure we import the local qmrpy (repo ./src) even if the notebook kernel is not started via `uv run`.
from pathlib import Path
import sys

def _find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / 'pyproject.toml').exists():
            return p
    return start

_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / 'src'
if _src.exists():
    sys.path.insert(0, str(_src))

import numpy as np

from qmrpy.models.t1 import VfaT1
from qmrpy.models.t2 import MonoT2
from qmrpy.sim import (
    SimCRLB,
    SimFisherMatrix,
    SimRnd,
    SimVary,
    add_gaussian_noise,
    add_rician_noise,
    crlb_cov_mean,
    optimize_protocol_grid,
    sensitivity_analysis,
    simulate_parameter_distribution,
    simulate_single_voxel,
)

rng = np.random.default_rng(0)
np.set_printoptions(precision=6, suppress=True)

In [ ]:
import sys, qmrpy
print('python:', sys.executable)
print('qmrpy :', qmrpy.__file__)
print('sys.path[0:3]:', sys.path[:3])


In [ ]:
# plotnine setup
from plotnine import (ggplot, aes, geom_point, geom_line, geom_histogram, geom_abline, geom_col, theme_bw, labs)
import pandas as pd


## 1) SingleVoxel: 単一ボクセルの信号生成→（任意）ノイズ付与→フィット

qMRLabのSingleVoxel GUI相当の“核”として、1セットのパラメータから信号を生成し、必要ならノイズを加えて、同じモデルでフィットし直します。


In [ ]:
vfa = VfaT1(flip_angle_deg=np.array([3.0, 8.0, 15.0, 25.0]), tr_s=0.015, b1=1.0)
params_true = {"m0": 2000.0, "t1_s": 1.0}

out = simulate_single_voxel(
    vfa,
    params=params_true,
    noise_model="gaussian",
    noise_snr=50.0,
    rng=rng,
    fit=True,
)

print('signal_clean:', out['signal_clean'])
print('signal      :', out['signal'])
print('fit         :', out['fit'])


### 可視化（plotnine）: flip angleごとの信号（clean/observed）


In [ ]:
df = pd.DataFrame({
    'flip_angle_deg': vfa.flip_angle_deg,
    'signal_clean': out['signal_clean'],
    'signal': out['signal'],
})
df_long = df.melt(id_vars=['flip_angle_deg'], value_vars=['signal_clean','signal'], var_name='kind', value_name='value')
(ggplot(df_long, aes('flip_angle_deg', 'value', color='kind'))
 + geom_line()
 + geom_point(size=2)
 + theme_bw()
 + labs(title='VFA T1: single-voxel signal', x='flip angle [deg]', y='signal'))


### ノイズ（2）は既に実装済み（Gaussian / Rician）
qMRLabのqMRIでは magnitude noise（Rician）が問題になることが多いので、両方のノイズを比較します。


In [ ]:
signal = out['signal_clean']
g = add_gaussian_noise(signal, sigma=10.0, rng=rng)
r = add_rician_noise(signal, sigma=10.0, rng=rng)

print('gaussian:', g)
print('rician  :', r)

### 可視化（plotnine）: ノイズ付与の比較（gaussian vs rician）


In [ ]:
dfn = pd.DataFrame({
    'flip_angle_deg': vfa.flip_angle_deg,
    'gaussian': g - signal,
    'rician': r - signal,
})
dfn_long = dfn.melt(id_vars=['flip_angle_deg'], value_vars=['gaussian','rician'], var_name='noise', value_name='delta')
(ggplot(dfn_long, aes('delta'))
 + geom_histogram(bins=20)
 + theme_bw()
 + labs(title='Noise delta histogram (value - clean)', x='delta', y='count'))


## 3) SimVary: 感度解析（1パラメータ掃引＋複数回ノイズ→フィット統計）

- `sensitivity_analysis` は“1つのパラメータだけ”を掃引するコア関数
- `SimVary` は qMRLab 互換のラッパ（複数パラメータを順に掃引）


In [ ]:
sens = sensitivity_analysis(
    vfa,
    nominal_params={"m0": 2000.0, "t1_s": 1.0},
    vary_param="t1_s",
    lb=0.7,
    ub=1.3,
    n_steps=7,
    n_runs=30,
    noise_model="gaussian",
    noise_snr=50.0,
    rng=rng,
)

print('x:', sens['x'])
print('t1_hat mean:', sens['mean']['t1_s'])
print('t1_hat std :', sens['std']['t1_s'])


### 可視化（plotnine）: 感度解析（x vs 推定平均）


In [ ]:
dfs = pd.DataFrame({
    't1_true': sens['x'],
    't1_hat_mean': sens['mean']['t1_s'],
    't1_hat_std': sens['std']['t1_s'],
})
(ggplot(dfs, aes('t1_true', 't1_hat_mean'))
 + geom_point()
 + geom_line()
 + geom_abline(intercept=0, slope=1)
 + theme_bw()
 + labs(title='Sensitivity (noise-free here): T1 hat vs T1 true', x='T1 true [s]', y='T1 hat mean [s]'))


qMRLab互換の `SimVary` を使う場合は、OptTable（fx/st/lb/ub/xnames）を渡します。


In [ ]:
from dataclasses import dataclass

@dataclass
class OptTable:
    xnames: list[str]
    fx: list[bool]
    st: list[float]
    lb: list[float]
    ub: list[float]

table = OptTable(
    xnames=["m0", "t1_s"],
    fx=[True, False],
    st=[2000.0, 1.0],
    lb=[2000.0, 0.7],
    ub=[2000.0, 1.3],
)

vary_all = SimVary(vfa, runs=5, OptTable=table, Opts={"SNR": 50})
print(vary_all.keys())
print('SimVary[t1_s].x:', vary_all['t1_s']['x'])


## 4) SimRnd: パラメータ分布（多ボクセル）→（任意）ノイズ→フィット→誤差統計

qMRLabのSimRndは、パラメータの分布（各ボクセルの真値）を与えて、推定誤差統計を返します。


In [ ]:
rnd = SimRnd(
    vfa,
    {
        'm0': rng.normal(2000.0, 0.0, size=100),
        't1_s': rng.uniform(0.7, 1.3, size=100),
    },
    {"SNR": 50},
)

print('RMSE keys:', rnd['RMSE'].keys())
print('RMSE[t1_s]:', rnd['RMSE']['t1_s'])
print('NRMSE[t1_s]:', rnd['NRMSE']['t1_s'])


### 可視化（plotnine）: SimRnd（推定 vs 真値）


In [ ]:
df_rnd = pd.DataFrame({
    't1_true': rnd['true']['t1_s'],
    't1_hat': rnd['hat']['t1_s'],
})
(ggplot(df_rnd, aes('t1_true', 't1_hat'))
 + geom_point(alpha=0.6)
 + geom_abline(intercept=0, slope=1)
 + theme_bw()
 + labs(title='SimRnd: T1 fitted vs true', x='T1 true [s]', y='T1 hat [s]'))


モデルを変える例（MonoT2）も同様に動きます。


In [ ]:
mono = MonoT2(te=np.array([10.0, 20.0, 40.0, 80.0]))
true_t2 = rng.uniform(30.0, 90.0, size=50)
dist = simulate_parameter_distribution(
    mono,
    true_params={"m0": 1000.0, "t2": true_t2},
    noise_model="rician",
    noise_snr=50.0,
    fit_kwargs={"fit_type": "exponential"},
)

print('metrics:', dist['metrics'])
print('t2 true head:', dist['true']['t2'][:5])
print('t2 hat  head:', dist['hat']['t2'][:5])


### 可視化（plotnine）: MonoT2（推定 vs 真値 / 残差ヒストグラム）


In [ ]:
df_t2 = pd.DataFrame({
    't2_true': dist['true']['t2'],
    't2_hat': dist['hat']['t2'],
})
df_t2['t2_err'] = df_t2['t2_hat'] - df_t2['t2_true']
p_scatter = (ggplot(df_t2, aes('t2_true', 't2_hat'))
 + geom_point(alpha=0.6)
 + geom_abline(intercept=0, slope=1)
 + theme_bw()
 + labs(title='MonoT2: fitted vs true', x='T2 true [ms]', y='T2 hat [ms]'))
p_hist = (ggplot(df_t2, aes('t2_err'))
 + geom_histogram(bins=20)
 + theme_bw()
 + labs(title='MonoT2: residual histogram (hat - true)', x='T2 error [ms]', y='count'))
p_scatter


In [ ]:
p_hist


## 5) SimProtocolOpt相当: Fisher情報→CRLB→目的関数（mean(diag(CRLB)/x^2)）

qMRLabのSimCRLB/SimFisherMatrixに合わせて、同名ラッパも用意しています。

注意: qMRLabでは `obj.xnames` / `obj.fx` を前提にしていますが、qmrpyのモデルは必ずしも保持しません。
そのため、ここでは `xnames` を持つ薄いproxyを使って評価します。


In [ ]:
class VfaProxy:
    xnames = ['m0', 't1_s']
    fx = [False, False]

    def __init__(self, model):
        self._m = model

    def forward(self, **params):
        return self._m.forward(**params)

    def fit_linear(self, signal, **kwargs):
        return self._m.fit_linear(signal, **kwargs)

proxy = VfaProxy(vfa)
x = np.array([2000.0, 1.0])

F = SimFisherMatrix(proxy, Prot=None, x=x, variables=[1, 2], sigma=1.0)
Fmean, names, CRLB, Fall = SimCRLB(proxy, Prot=None, xvalues=x[None, :], sigma=1.0)

print('Fisher:')
print(F)
print('CRLB:')
print(CRLB)
print('objective mean(diag(CRLB)/x^2):', Fmean)
print('xnames:', names)

### 可視化（plotnine）: CRLBの対角（diag(CRLB)/x^2）


In [ ]:
diag_cov = np.diag(CRLB) / (x ** 2)
df_crlb = pd.DataFrame({'param': names, 'diag_crlb_over_x2': diag_cov})
(ggplot(df_crlb, aes('param', 'diag_crlb_over_x2'))
 + geom_col()
 + theme_bw()
 + labs(title='CRLB proxy per parameter', x='parameter', y='diag(CRLB)/x^2'))


### プロトコル候補のグリッド探索（簡易版）
ここではVFAのflip angleセットを候補として列挙し、CRLB目的関数が小さいものを選びます。


In [ ]:
params = {'m0': 2000.0, 't1_s': 1.0}
candidates = [
    np.array([3.0, 8.0, 15.0, 25.0]),
    np.array([2.0, 5.0, 10.0, 18.0]),
    np.array([5.0, 10.0, 20.0, 30.0]),
]

out = optimize_protocol_grid(
    lambda fa: VfaT1(flip_angle_deg=fa, tr_s=0.015, b1=1.0),
    protocol_candidates=candidates,
    params=params,
    variables=['m0', 't1_s'],
    sigma=1.0,
)

print('best_index:', out['best_index'])
print('best_protocol:', out['best_protocol'])
print('objectives:', np.array(out['objectives']))

# 参考: 直接 objective を呼ぶ（qMRLabのSimCRLB相当のスカラー）
for fa in candidates:
    m = VfaT1(flip_angle_deg=fa, tr_s=0.015, b1=1.0)
    print(fa, '->', crlb_cov_mean(m, params=params, variables=['m0','t1_s'], sigma=1.0))

### 可視化（plotnine）: プロトコル候補ごとの目的関数（小さいほど良い）


In [ ]:
df_obj = pd.DataFrame({
    'candidate': list(range(len(candidates))),
    'objective': out['objectives'],
})
(ggplot(df_obj, aes('candidate', 'objective'))
 + geom_col()
 + theme_bw()
 + labs(title='Protocol grid: objective by candidate', x='candidate index', y='mean(diag(CRLB)/x^2)'))


---

必要なら次の拡張ができます:
- SNR指定をqMRLabの定義にさらに寄せる（信号スケール依存の扱いなど）
- プロトコルの連続最適化（グリッドではなく最適化アルゴリズム）
- plotnine/matplotlib で可視化を追加
